In [ ]:
#https://anaconda.org/anaconda/sas7bdat
from sas7bdat import SAS7BDAT
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np


In [ ]:
with SAS7BDAT('crime.sas7bdat') as file:
    df_sas=file.to_data_frame()

In [ ]:
#print(df_sas['Location'])
print(df_sas[['Description','IUCR']])
print(df_sas.columns)

In [ ]:
pd.DataFrame.hist(df_sas[['Year']])
plt.ylabel('count')
plt.show()

df_sas['ArrestInt']=df_sas['Arrest'].apply(lambda x: 0 if x=='true' else 1)

#print(df_sas['ArrestInt'])
plt.figure(figsize=(22,6))
iucr=df_sas.groupby(['IUCR'])['ArrestInt'].sum().nlargest(40).sort_values(ascending=False)
iucr.plot.bar()
plt.show()

#print(iucr.index)

_ = sns.swarmplot(y=iucr.values)
_ = plt.xlabel('IUCR')
_ = plt.ylabel('Arrest by IUCR code')

# Show the plot
plt.show()

#print(df_sas.groupby(['District'])['Arrest'].count())

In [ ]:
arrest_descriptions=df_sas[df_sas['IUCR'].isin(iucr.index)]['Description'].unique()

arrest_descriptions=np.sort(arrest_descriptions)

result= df_sas[df_sas['Description'].isin(arrest_descriptions)]

#result=result[result['Year']==2016].groupby('Description')['ArrestInt'].apply(lambda x: x.sum()).reset_index()
result=result.groupby('Description')['ArrestInt'].apply(lambda x: x.sum()).reset_index()

#print(result.head())

print (df_sas['Year'].unique())

for key,item in result.iterrows():
    print(item['Description']," [Occurrences: "+ str(item['ArrestInt'])+']')
    #print(item.Description,item.ArrestInt)

In [ ]:
#print(df_sas['Domestic'])

df_sas['DomesticInt']=df_sas['Domestic'].astype(bool).astype(int)
#result=result.groupby('Description')['Domestic'].apply(lambda x: x.sum()).reset_index()

plt.figure(figsize=(22,6))
iucr=df_sas.groupby(['IUCR'])['DomesticInt'].sum().nlargest(40).sort_values(ascending=False)
iucr.plot.bar()
plt.show()


In [ ]:
#features = ['IUCR','fbi_code','arrest_code','primary_type']

#sns.pairplot(
#    data=df_sas,
#    vars=features,
#    size=1,
#    plot_kws={'s': 5}
#)
#plt.show()
#print("arrest_code")
#print(df_sas['arrest_code'].value_counts())
#print("fbi_code")

df_sas['Domestic']=df_sas['Domestic'].astype(bool)

filter=df_sas['Domestic']==True
print("Domestic:")
#print(df_sas[filter]['fbi_code'].value_counts())
df_sas[filter]['fbi_code'].value_counts().plot.bar()
plt.show()

filter=df_sas['Domestic'].isnull()==True

print("no Non-Domestic:")
#print(df_sas[filter]['fbi_code'].value_counts())
#result=df_sas[filter]['fbi_code'].value_counts()
#print(len(result))
#plt.show()

plt.figure(figsize=(22,6))
print("IUCR")
#print(df_sas['IUCR'].value_counts())
df_sas['IUCR'].value_counts().nlargest(100).plot.bar()
plt.show()

In [ ]:
districts=df_sas['District'].unique()
print(districts)

print(df_sas[df_sas['District'].isin(districts)]['ArrestInt'].mean())

print(df_sas.groupby('District')['ArrestInt'].mean())

print(df_sas.groupby(['District','Ward'])['ArrestInt'].mean())

In [ ]:
def left(s, amount):
    return s[:amount]

def right(s, amount):
    return s[-amount:]

def mid(s, offset, amount):
    return s[offset:offset+amount]

df_sas['Year']=pd.DatetimeIndex(df_sas['date']).year
df_sas['Hour']=pd.DatetimeIndex(df_sas['date']).hour
df_sas['Month']=pd.DatetimeIndex(df_sas['date']).month

arrest_year=df_sas.groupby(['Year'])['ArrestInt'].sum()

plt.figure(figsize=(22,6))
arrest_year.plot()
plt.xlabel('Year')
plt.ylabel('Arrests')
plt.show()

In [ ]:
fbi_codes=df_sas['fbi_code'].unique()

for code in fbi_codes:
    #print(code)
    filter=df_sas['fbi_code']==code
    data=df_sas[filter].groupby(['Year','Month'])['ArrestInt'].sum()
    plt.figure(figsize=(12,2))             
    plt.title(code)
    data.plot()
    plt.show()




In [ ]:
result=df_sas.groupby(['Year','Month','fbi_code'])['ArrestInt'].sum().reset_index()
#print(top20.columns)
mask=result['ArrestInt']>30
fbi_codes=result[mask]['fbi_code'].unique()

filter=df_sas['fbi_code'].isin(fbi_codes) 
fbi_codes=df_sas['fbi_code'].unique()

arrest_breakdown=df_sas[filter].groupby(['Year','Month','fbi_code'])['ArrestInt'].sum().reset_index()
keys=arrest_breakdown.keys()
#print(arrest_breakdown)

g = sns.factorplot(data=arrest_breakdown, x='Year', y='ArrestInt', 
                  hue='fbi_code',  kind='point',size=8,aspect=2)

plt.show()



In [ ]:
def check(polygon, lon, lat):
    # build a shapely point from your geopoint
    point = Point(lon, lat)

    # the contains function does exactly what you want
    return polygon.contains(point)

import shapefile
from shapely.geometry import shape, Point,Polygon

myshp = open('idaho_shapely.shp', 'rb')
mydbf = open('idaho_shapely.dbf', 'rb')

reader = shapefile.Reader(shp=myshp, dbf=mydbf)

print(reader.bbox)
print(reader.numRecords)
#print(type(shape))

#shapes = reader.shapes()
#polygon = shape(shapes[0])  

#print(polygon)

#first feature of the shapefile
#feature = shape.shapeRecords()[0]
#first = feature.shape.__geo_interface__  
#print (first)

#'X_Coordinate', 'Y_Coordinate'
df_idaho=df_sas.copy(deep=False)
for index,item in df_sas.iterrows():
    #print(item)
    #gps=  list(eval(item['Location']))
    #print(gps)
    latitude=item['Latitude']
    longitude=item['Longitude']
    for i in np.arange(len(shapes)):
        polygon=shape(shapes[i])
        if check(polygon, latitude,longitude)==True:
            df_idaho.append(item)
            print(item)
    #print(latitude,longitude)
    

[-117.046587, 41.998903, -111.080857, 48.735223999999995]
227


In [ ]:
print(df_idaho.head())